In [324]:
zdb_root.artists_db._p_changed

False

In [13]:
# import ZODB
# import transaction

# class ArtistZDB:
    
#     def __init__(self, file_name="RTArtists.fs"):
#         self.db = ZODB.DB(file_name)
#         self.connection = None
#         self.root = None
#         self.opened = False
    
#     def open_db(self):
#         if self.opened:
#             return
#         try:
#             self.connection = self.db.open()
#             self.root = self.connection.root
#         except Exception as ex:
#             print(ex)
#             self.connection = db.open()
#             self.zdb_root = connection.root
#         self.opened = True
            
#     def save(self):
#         if self.opened:
#             transaction.commit()
    
#     def undo(self):
#         if self.opened:
#             transaction.abort()
    
#     def close_db(self):
#         if self.opened:
#             transaction.commit()
#             self.db.close()
#             self.opened = False

# def open_zdb(name="IansArtists.fs"):
#     try:
#         db = ZODB.DB("IansArtists.fs")
#         connection = db.open()
#         zdb_root = connection.root
#         return zdb_root
#     except Exception as ex:
#         print(ex)

# def save_zdb():
#     transaction.commit()

# def close_zdb():
#     db.close()
    
# def new_zdb(name="IansArtists2.fs"):
#     db = ZODB.DB("IansArtists.fs")
#     connection = db.open()
#     zdb_root = connection.root
#     zdb_root.artists_db = RTArtists()
#     return zdb_root


In [181]:
transaction.abort()

In [182]:
db.close_db()
db = ArtistZDB(file_name="IansArtists2.fs")
db.open_db()
zdb_root = db.root
dir(db.root.artists_db)
artists_db = zdb_root.artists_db

In [ ]:
len(zdb_root2.artists_db.keys())

In [206]:
db2 = ZODB.DB("IansArtists3.fs")
connection2 = db2.open()
zdb_root2 = connection2.root
zdb_root2.artists_db = artists_db
db2.close()

In [229]:
db2.close()

In [ ]:
import json
json.dumps(art_obj)

In [266]:
from dataclasses import dataclass, field
from datetime import datetime
from enum import Enum, auto
import uuid
from typing import Dict, List, Any


def _parse_enum(enum_type, name: str):
    cf_members = {n.casefold(): v for n,v in enum_type.__members__.items()}
    if name.casefold() in cf_members:
        return cf_members[name.casefold()]
    return None

class ArtistRating(Enum):
    Other = auto()
    Favorite = auto()
    Like = auto()
    DontLike = auto()
    Unknown = auto()

    @classmethod
    def parse(cls, name: str):
        enum_val = _parse_enum(cls, name)
        return enum_val if enum_val else cls.Other

class ArtistType(Enum):
    Other = auto()
    Person = auto()
    Group = auto()

    @classmethod
    def parse(cls, name: str):
        enum_val = _parse_enum(cls, name)
        return enum_val if enum_val else cls.Other
    

class RTArtists():
    
    def __init__(self):
        self.artists: dict = dict()
        self.artist_ids: dict = dict()
        self.artist_spids: dict = dict()
            
    def add_artist(self, artist):
        self.artists[artist.name] = artist
        if artist.mbid:
            self.artist_ids[artist.mbid] = artist
        if artist.spid:
            self.artist_spids[artist.spid] = artist

    def del_artist(self, artist):
        if isinstance(artist, RTArtist):
            key = artist.name
        elif isinstance(artist, str):
            key = artist
            
        if key in self.artists:
            del_artist = self.artists.pop(key)
            if del_artist.mbid in self.artist_ids:
                self.artist_ids.pop(del_artist.mbid)
        elif key in self.artist_ids:
            del_artist = self.artist_ids.pop(key)
            if del_artist.name in self.artists:
                self.artists.pop(del_artist.name)

    def __getitem__(self, key):
        if key in self.artists:
            return self.artists[key]
        if key in self.artist_ids:
            return self.artist_ids[key]
        return None
        
    def __contains__(self, key):
        return key in self.artists or key in self.artist_ids
    
    def add_artists(self, artists):
        for artist in artists:
            self.add_artist(artist)

@dataclass
class RTArtist():
    name: str
    sort_name: str = None
    mbid: str = None
    spid: str = None
    discogs_id: str = None
    artist_type: ArtistType = ArtistType.Other
    artist_rank: int = 500
    favorite: bool = False
    lfm_url: str = None
    discogs_url: str = None
    bio_summary: str = None
    bio_content: str = None
    tags: list = field(default_factory=list)
    artist_rating: ArtistRating = ArtistRating.Unknown
    releases: dict = field(default_factory=dict)
    aliases: list = field(default_factory=list)
    name_variations: list = field(default_factory=list)
    members: dict = field(default_factory=dict)
    member_of: dict = field(default_factory=dict)
    similar: dict = field(default_factory=dict)
    ext_data: dict = field(default_factory=dict)
    img_url: str = None
    tracked: bool = False
    
    @property
    def musicbrainz_url(self):
        if self.mbid:
            return f"https://musicbrainz.org/artist/{self.mbid}"
        return None
    
    def has_data(self, key):
        return key.casefold() in self.ext_data
    
    def _set_ext_data(self, key, data, overwrite=True):
        if key.casefold() in ["lfm_artist", "discogs_artist", "mb_artist"]:
            self.ext_data[key.casefold()] = data
        else:
            raise KeyError(f"Unknown data type {key}")
    
    def populate_discogs_data(self, overwrite=False):
        if not overwrite and self.has_data("discogs_artist"):
            return
        if self.discogs_id:
            artist = discogs_get_artist_by_id(self.discogs_id)
        else:
            artist = discogs_find_artist(self.name)
        if artist:
            self._set_ext_data("discogs_artist", artist)
            self.discogs_url = f"https://www.discogs.com/{artist.discogs_id}"
            self.aliases = [a.name for a in artist.aliases]
            self.name_variations = list(artist.name_variations)
            if artist.images and "uri" in artist.images[0]:
                self.img_url = artist.images[0]["uri"] 
    
    def populate_mb_data(self, overwrite=False):
        if not overwrite and self.has_data("mb_artist"):
            return
        if not self.mbid:
            return
        artist = mb_get_artist_by_id(self.mbid)
        if not artist:
            return
        self.artist_type = ArtistType.parse(artist.get("type"))
        self.sort_name = artist.get("sort-name")
        self.discogs_id = mb_get_discogs_id(artist)
        self._set_ext_data("mb_artist", artist)
        membership = mb_get_member_of(artist)
        for member in membership:
            member_artist = RTArtist(name = member["name"], mbid=member["id"])
            if self.artist_type == ArtistType.Person:
                self.member_of[member_artist.name] = member_artist
            elif self.artist_type == ArtistType.Group:
                self.members[member_artist.name] = member_artist

    def populate_lfm_data(self, overwrite=False):
        if not overwrite and self.has_data("lfm_artist"):
            return
        artist = lfm_get_artist_info(self.name, LFM_KEY)
        if not artist:
            return
        print(list(self.ext_data.keys()))
        self._set_ext_data("lfm_artist", artist)
        self.mbid = artist.get("mbid")
        self.lfm_url = artist.get("url")
        bio = lfm_artist_bio(artist)
        if bio:
            self.bio_summary = bio.get("summary")
            self.bio_content = bio.get("content")
        tag_list = artist.get("tags", {}).get("tag", [])
        tag_names = [t_item["name"] for t_item in tag_list]
        self.tags.append(tag_names)

    def get_similar_artists(self, artist_db=None, overwrite=False):
        if not overwrite and self.similar:
            return
        similar_artists_raw = lfm_get_similar_artists(
            artist=self.name, api_key=LFM_KEY, num_artists=40
        )
        similar_artists = {
            name: s_artist for name, s_artist 
            in similar_artists_raw.items() 
            if float(s_artist["match"]) >= 0.125
        }
        for a_name, s_artist in similar_artists.items():
            if artist_db:
                if a_name in artist_db.artists:
                    print("linking existing artist:", a_name)
                    self.similar[a_name] = artist_db[a_name]
                    continue
            self.similar[a_name] = RTArtist(
                name=a_name, mbid=s_artist.get("mbid"), lfm_url=s_artist.get("url"))

    def _add_mb_releases(self):
        if "mb_artist" in self.ext_data:
            releases = mb_get_group_releases(self.ext_data["mb_artist"])
            for release in releases:
                self.releases[release.title] = release

    def _create_mb_release(self, release) -> RTRelease:
        rel_date = release.get("first-release-date")
        release = RTRelease(
            title = release.get("title"),
            mbid = release.get("id"),
            primary_type = release.get("primary_type"),
            other_type = release.get("type"),
            release_date = rel_date if isinstance(rel_date, datetime) else None,
        )
        release.set_ext_data("mb_release", release)
        return release

@dataclass
class RTRelease():
    title: str
    mbid: str = None
    primary_type: str = None
    other_type: str = None
    release_date: datetime = None
    ext_data: dict = field(default_factory=dict)

    def set_ext_data(self, key, data, overwrite=True):
        if key.casefold() in ["lfm_release", "discogs_release", "mb_release"]:
            self.ext_data[key.casefold()] = data
        else:
            raise KeyError(f"Unknown data type {key}")

In [254]:
# Write pickle
import pickle
from pathlib import Path
import random

def save_db_to_pkl(db, file_name):
    target_path = Path(file_name).resolve()
    if target_path.is_file():
        temp_path = Path(str(target_path) + ".tmp")
        target_path.replace(temp_path)    
    try:
        with open(target_path, "wb") as db_file:
            pickle.dump(db, db_file)
    except (pickle.PickleError, IOError):
        temp_path.rename(target_path)
        print("Failed to write new file")
        
def load_db_from_pkl(file_name):
    target_path = Path(file_name)
    
    try:
        with open(target_path, "rb") as db_file:
            db_obj = pickle.load(db_file)
            return db_obj
    except (pickle.PickleError, IOError):
        temp_path.rename(target_path)
        print("Failed to read db file")
    return None


In [257]:

# art_x = load_db_from_pkl("artists2.pkl")
# art_x.artists.keys()

dict_keys(['Ian Hellen', 'Jack White', 'Beck', 'Brian Eno', 'Queens of the Stone Age', 'Bruno Sanfilippo', 'Bonobo', 'The Strokes', 'Beirut', 'Eagles of Death Metal', 'Lana Del Rey', 'Rafe Pearlman & Jonathan Plum', 'Ólafur Arnalds', 'Courtney Barnett', 'Janelle Monáe', 'David Bowie', 'Scott Walker', 'The Chemical Brothers', 'Lou Reed', 'The Kooks', 'Cage the Elephant', 'Mac DeMarco', 'The Velvet Underground', 'The White Stripes', 'Turin Brakes', 'Los Lobos', 'Max Richter', 'Thom Yorke', 'Bastille', 'Nils Frahm', 'Field Rotation', 'Zero 7', 'Hauschka', 'Django Django', 'Erik Satie', 'Flying Lotus', 'Joywave', 'The Heavy', 'Blur', 'Oh Wonder', 'Boards of Canada', 'MF DOOM', 'Ice Cube', 'Purity Ring', '坂本龍一', 'Lorde', 'Richard H. Kirk', 'Gorillaz', 'Morcheeba', 'Cake', 'Jóhann Jóhannsson', 'The Prodigy', 'Years & Years', 'Arctic Monkeys', "Dustin O'Halloran", 'elbow', 'Damon Albarn', 'Eno • Hyde', 'Federico Aubele', 'Neutral Milk Hotel', 'Robert Rich', 'Bomba Estéreo', 'Bruno Sanfilippo 

In [258]:
artists_db2.artists["Jack White"]
"3ae2fb37-8a23-429d-9920-bac811c4fc22" in artists_db2

True

In [78]:
jwhite = zdb_root.artists_db["Jack White"]
jwhite.populate_lfm_data()
jwhite.populate_mb_data()
jwhite.populate_discogs_data()
jwhite

RTArtist(name='Jack White', sort_name='White, Jack', mbid='3ae2fb37-8a23-429d-9920-bac811c4fc22', discogs_id='278763', artist_type=<ArtistType.Person: 2>, artist_rank=2, favorite=False, lfm_url='https://www.last.fm/music/Jack+White', discogs_url='https://www.discogs.com/https://www.discogs.com/artist/278763-Jack-White-2', bio_summary='\nJack White (born John Anthony Gillis on July 9, 1975 in Detroit), American rock musician, producer, and actor. He is best known as the guitarist and lead vocalist of the rock duo The White Stripes (1997-2011) and more recently guitarist and vocalist of The Raconteurs, and drummer and vocalist for the band The Dead Weather.\n\nJack White first played as a professional musician in the early 1990s, as a drummer for the Detroit cowpunk band Goober & the Peas. <a href="https://www.last.fm/music/Jack+White">Read more on Last.fm</a>', bio_content='\nJack White (born John Anthony Gillis on July 9, 1975 in Detroit), American rock musician, producer, and actor. H

# Initial Tracking List

In [231]:
def create_artist(row):
    artist = RTArtist(name=row["name"], mbid=row["mbid"], lfm_url=row["url"], artist_rank=int(row["@attr.rank"]))
    if "tracked" in row:
        artist.tracked = row["tracked"]
    return artist

top_artists_df = lfm_get_top_artists_df(user="ianhellen", limit=200)
# list(top_artists_df.apply(create_artist, axis=1).array)[0]

top_artists_df.head()

,mbid,url,playcount,image,name,streamable,@attr.rank
0,,https://www.last.fm/music/Ian+Hellen,155,"[{'size': 'small', '#text': 'https://lastfm.fr...",Ian Hellen,0,1
1,3ae2fb37-8a23-429d-9920-bac811c4fc22,https://www.last.fm/music/Jack+White,153,"[{'size': 'small', '#text': 'https://lastfm.fr...",Jack White,0,2
2,,https://www.last.fm/music/Beck,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Beck,0,3
3,ff95eb47-41c4-4f7f-a104-cdc30f02e872,https://www.last.fm/music/Brian+Eno,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Brian Eno,0,4
4,7dc8f5bd-9d0b-4087-9f73-dc164950bbd8,https://www.last.fm/music/Queens+of+the+Stone+Age,85,"[{'size': 'small', '#text': 'https://lastfm.fr...",Queens of the Stone Age,0,5


In [233]:
tracked_select = widgets.SelectMultiple(
    description="Tracked",
    options=top_artists_df["name"].array,
    layout=widgets.Layout(height="300px", width="300px")
)
untrack_btn = widgets.Button(description="Move to Untracked")
track_btn = widgets.Button(description="Move to Tracked")
untracked_select = widgets.Select(
    description="Not Tracked",
    options=[],
    layout=widgets.Layout(height="300px", width="300px")
)

def move_to_untracked(evt):
    del evt
    curr_list = set(untracked_select.options)
    curr_list.update(tracked_select.value)
    untracked_select.options = sorted(curr_list)
    tracked_select.options = [a for a 
                              in tracked_select.options 
                              if a not in untracked_select.options]
def move_to_tracked(evt):
    del evt
    curr_list = set(tracked_select.options)
    curr_list.update(untracked_select.value)
    to_delete_select.options = sorted(curr_list)
    tracked_select.options = [a for a 
                               in untracked_select.options 
                               if a not in tracked_select.options]
    
untrack_btn.on_click(move_to_untracked)
track_btn.on_click(move_to_tracked)
buttons = widgets.VBox([untrack_btn, track_btn])
display(widgets.HBox([tracked_select, buttons, untracked_select]))

In [234]:
top_artists_df["tracked"] = ~ top_artists_df["name"].isin(untracked_select.options)

## Add Tracked artists to DB

In [260]:
artists_db = RTArtists()
artists_db.add_artists(top_artists_df.apply(create_artist, axis=1).array)
DB_FILE_NAME = "artists_db.pkl"

In [267]:
artists_db = load_db_from_pkl(DB_FILE_NAME)

In [294]:
save_db_to_pkl(artists_db, DB_FILE_NAME)

In [277]:
new_artist = RTArtist(**vars(list(artists_db.artists.values())[1]))
new_artist

RTArtist(name='Jack White', sort_name=None, mbid='3ae2fb37-8a23-429d-9920-bac811c4fc22', spid=None, discogs_id=None, artist_type=<ArtistType.Other: 1>, artist_rank=2, favorite=False, lfm_url='https://www.last.fm/music/Jack+White', discogs_url=None, bio_summary=None, bio_content=None, tags=[], artist_rating=<ArtistRating.Unknown: 5>, releases={}, aliases=[], name_variations=[], members={}, member_of={}, similar={}, ext_data={}, img_url=None, tracked=True)

# Display

In [287]:
import ipywidgets as widgets
from IPython.display import Image, display, Markdown
import pdb



def display_artist(selection):
    if (
        selection is None
        or "new" not in selection
        #or not isinstance(selection["new"], int)
    ):
        return
    artist_name = selection["new"]
    
    artist_rec = artists_db[artist_name]
    artist_mbid = artist_rec.mbid
    artist_output.clear_output()
    with artist_output:
        print(artist_name)
        display(Markdown(f"getting data..."))
    
    artist_rec.populate_mb_data()
    new_artists, known_artists = get_new_sim_artists(artists_db, artist_rec)

    artist_output.clear_output()
    fav_check.value = artist_rec.favorite
    art_rating.value = artist_rec.artist_rating.name
    tracked_check.value = artist_rec.tracked
    
    with artist_output:
        display(Markdown(f"## {artist_name}"))
        display(Markdown(f"MusicBrainz mbid: {artist_mbid} URL: {artist_rec.musicbrainz_url}"))
        display(Markdown(f"Discogs URL: {artist_rec.discogs_url}"))
        if artist_rec.img_url:
            display(Image(url=artist_rec.img_url))
        display(Markdown(f"LastFM URL: {artist_rec.lfm_url}"))
        display(Markdown(f"MusicBrainz "))
        
    display_artist_data()

def fetch_artist_data(evt):
    del evt
    artist_rec = get_selected_artist()
    artist_rec.populate_lfm_data()
    artist_rec.populate_mb_data()
    artist_rec.populate_discogs_data()
    if artist_rec.tracked:
        artist_rec.get_similar_artists(artists_db)
    display_artist_data()

def display_artist_data():
    artist_rec = get_selected_artist()
    new_artists, known_artists = get_new_sim_artists(artists_db, artist_rec)
    with artist_output:
        display(Markdown(artist_rec.bio_summary))
        if artist_rec.img_url:
            display(Image(url=artist_rec.img_url))
        display(Markdown(f"LastFM URL: {artist_rec.lfm_url}"))
        
        if artist_rec.aliases:
            display(Markdown("#### Aliases"))
            display(Markdown(", ".join(artist_rec.aliases)))
        if artist_rec.name_variations:
            display(Markdown("#### Name variants"))
            display(Markdown(", ".join(artist_rec.name_variations)))
        if artist_rec.artist_type == ArtistType.Person:
            display(Markdown("#### Member of"))
            display(Markdown(", ".join(artist_rec.member_of.keys())))
        elif artist_rec.artist_type == ArtistType.Group:
            display(Markdown("#### Members"))
            display(Markdown(", ".join(artist_rec.members.keys())))
        display(Markdown("#### Related Artists (new)"))
        display(Markdown(f"**{', '.join(new_artists)}**"))
        display(Markdown("#### Related Artists (known)"))
        display(Markdown(f"*{', '.join(known_artists)}*"))
        
    similar_art_sel.options = new_artists
    visible = "visible" if bool(new_artists) else "hidden"
    similar_art_sel.visibility = visible
    artists_to_add.visibility = visible
    move_to_selected_btn.visibility = visible
    add_artists_btn.visibility = visible

def get_new_sim_artists(artists_db: RTArtists, current_artist):
    known_artists = set(artists_db.artists.keys())
    similar_artists = set(current_artist.similar.keys())
    new_artists = similar_artists - known_artists
    known_artists = known_artists & similar_artists
    return new_artists, known_artists


def move_to_selected(btn_event):
    del btn_event
    cur_selection = list(artists_to_add.options)
    artists_to_add.options=sorted(cur_selection + list(similar_art_sel.value))

def add_new_artists(btn_event):
    del btn_event
    cur_artist = get_selected_artist()
    for new_artist in artists_to_add.options:
        print(artists_to_add.options)
        sim_artist = cur_artist.similar.get(new_artist)
        artist_rec = RTArtist(
            name=new_artist,
            mbid=sim_artist.mbid if sim_artist else None,
            artist_rating = ArtistRating.Like,
            tracked=True)
        artist_rec.populate_lfm_data()
        artist_rec.populate_mb_data()
        artist_rec.populate_discogs_data()
        artist_rec.get_similar_artists(zdb_root.artists_db)
        artists_db.add_artist(artist_rec)
        #print(artist_rec)
    print(sort_opt.value)
    artists_to_add.options=[]

def change_sort_order(opt_event):
    if opt_event["new"] == "Rank":
        artist_list = get_artist_list(artists_db, sort=1)
    else:
        artist_list = get_artist_list(artists_db, sort=1)
    artist_select.options = artist_list

def set_favorite(evt):
    print(evt)
    del evt
    cur_artist = get_selected_artist()
    cur_artist.favorite = fav_check.value
    
def set_rating(evt):
    print(evt)
    del evt
    rating = ArtistRating.parse(art_rating.value)
    cur_artist = get_selected_artist()
    if cur_artist:
        cur_artist.artist_rating = rating

def get_selected_artist():
    if artist_select.label:
        return artists_db[artist_select.label]

def show_hide_tracked(evt):
    del evt
    artist_select.options = get_artist_list(artists_db, get_sort_key())

def get_sort_key():
    return 1 if sort_opt.value == "Rank" else 0

def set_tracked(evt):
    cur_artist = get_selected_artist()
    print(evt)
    if cur_artist:
        cur_artist.tracked = fav_check.value


def save_changes(evt):
    del evt
    save_db_to_pkl(db=artists_db, file_name=DB_FILE_NAME)

def open_close_db():
    global artists_db, db_opened
    return
#     if db_opened:
#         close_zdb()
#         open_close_btn.description = "Open Database"
#     else:
#         artists_db = open_zdb()
#         open_close_btn.description = "Close Database"
#     db_opened = not db_opened

def get_artist_list(artists_db=artists_db, sort=1):
    art_dict = {a.name: a.artist_rank
                for a in artists_db.artists.values()
                if show_tracked_check.value or a.tracked}
    return sorted(art_dict.items(), key=lambda x: x[sort])

# DB/Global Controls
save_btn = widgets.Button(description="Save to Database")
save_btn.on_click(save_changes)
fetch_data_btn = widgets.Button(description="Fetch Artist Data")
fetch_data_btn.on_click(fetch_artist_data)

sort_opt = widgets.RadioButtons(options=["Rank", "Name"], description="Sort by")
sort_opt.observe(change_sort_order, names="value")
show_tracked_check = widgets.Checkbox(description="Show Untracked", value=False)
show_tracked_check.observe(show_hide_tracked, names="value")

# Get Artists
artist_output = widgets.Output(layout={"border": "1px solid black"})
# art_dict = {a.name: a.artist_rank for a in artists_db.artists_db.artists.values()}
# art_dict = {v: k for k,v in top_artists_df["name"].to_dict().items()}

artist_select = widgets.Select(options=get_artist_list(artists_db, sort=1), 
                                  description="Current Artists",
                                  layout=widgets.Layout(width="30%", height="300px"),)

artist_select.observe(display_artist, names="label")


# Artist property controls
fav_check = widgets.Checkbox(description="Favorite", value=False)
fav_check.observe(set_favorite, names="value")
art_rating = widgets.Select(description="Rating", options=list(ArtistRating.__members__.keys()))
art_rating.observe(set_rating, names="value")
tracked_check = widgets.Checkbox(description="Tracked", value=False)
tracked_check.observe(set_tracked, names="value")

# Similar Artist selection
layout_hidden = widgets.Layout(visibility="hidden")
move_to_selected_btn = widgets.Button(description="Add Selection", layout=layout_hidden)
move_to_selected_btn.visible = False
move_to_selected_btn.on_click(move_to_selected)
add_artists_btn = widgets.Button(description="Add Selected to Tracked", layout=layout_hidden)
add_artists_btn.visible = False
add_artists_btn.on_click(add_new_artists)

similar_art_sel = widgets.SelectMultiple(options=[], 
                                         description="Similar Artists",
                                         layout=widgets.Layout(width="300px", height="200px", visibility="hidden"),
                                         style={"description_width": "initial"},
                                         )
artists_to_add = widgets.Select(options=[], 
                                description="Selected",
                                layout=widgets.Layout(width="300px", height="300px", visibility="hidden"),
                                style={"description_width": "initial"},
                                visible = False)

# Display
global_controls = widgets.VBox([fetch_data_btn, sort_opt, show_tracked_check, save_btn])
sim_artists_group = widgets.VBox([similar_art_sel, move_to_selected_btn])
selected_artists_group = widgets.VBox([artists_to_add, add_artists_btn])
artist_props_sub = widgets.VBox([fav_check, art_rating, tracked_check])
artist_props = widgets.HBox([artist_props_sub, sim_artists_group, selected_artists_group])
display(widgets.VBox([widgets.HBox([artist_select, global_controls]),
                      artist_output,
                      artist_props]))

In [292]:
bonobo = artists_db["Bonobo"]

In [293]:
%%debug
bonobo.populate_mb_data()

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()



ipdb>  s


--Call--
> <ipython-input-266-de2399f72f45>(136)populate_mb_data()
    134                 self.img_url = artist.images[0]["uri"]
    135 
--> 136     def populate_mb_data(self, overwrite=False):
    137         if not overwrite and self.has_data("mb_artist"):
    138             return



ipdb>  n


> <ipython-input-266-de2399f72f45>(137)populate_mb_data()
    135 
    136     def populate_mb_data(self, overwrite=False):
--> 137         if not overwrite and self.has_data("mb_artist"):
    138             return
    139         if not self.mbid:



...

ipdb>  eno.discogs_id


'634'


ipdb>  beck.ext_data["mb_artist"]


{'id': 'a8baaa41-50f1-4f63-979e-717c14979dfb', 'type': 'Person', 'name': 'Beck', 'sort-name': 'Beck', 'disambiguation': 'Danish rapper', 'gender': 'Male', 'country': 'DK', 'area': {'id': '4757b525-2a60-324a-b060-578765d2c993', 'name': 'Denmark', 'sort-name': 'Denmark', 'iso-3166-1-code-list': ['DK']}, 'release-group-list': [], 'release-group-count': 0}


ipdb>  q


# Discogs

In [76]:
import discogs_client 
do_clnt = discogs_client.Client('IansReleaseTracker/0.1', user_token="OGPWPJrEzRHgmLDqijnWgwRtbZYezeIWoeyPitNC")
from discogs_client import Artist as DOArtist

def discogs_find_artist(srch_artist, max_checks=0, max_pages=1, case_sensitive=False):
    artists = do_clnt.search(query=srch_artist, type="artist")
    count = 0
    for i in range(0, max_pages):
        if len(artists.page(i)) == 1:
            return artists.page(i)[0]
        for artist in artists.page(i):
            if case_sensitive and artist.name == srch_artist:
                return artist
            elif (not case_sensitive 
                  and artist.name.lower() == srch_artist.lower()):
                return artist
            count += 1
            if max_checks and count > max_checks:
                print(f"{srch_artist} not found")
                return None
            
def discogs_get_artist_by_id(mbid):
    return do_clnt.artist(mbid)

artist = discogs_find_artist("Jack White (2)")
print(artist)

<Artist 278763 'Jack White (2)'>


# MusicBrainz

In [26]:
!pip install musicbrainzngs
!pip install python-dateutil

In [77]:
import musicbrainzngs as mbrainz
from dateutil import parser as du_parser
mbrainz.set_useragent(app="IanReleaseTracker", version=0.1, contact="ianhellen@outlook.com")

jw_artist_id = "3ae2fb37-8a23-429d-9920-bac811c4fc22"
# result = musicbrainzngs.get_artist_by_id(artist_id,
def mb_get_artist_by_id(artist_id):
    mb_includes=["artist-rels", "release-groups", "url-rels"]
    try:
        result = mbrainz.get_artist_by_id(artist_id, includes=mb_includes)
    except WebServiceError as exc:
        print("Something went wrong with the request: %s" % exc)
    return result["artist"]


def mb_get_discogs_id(mb_artist):
    DICT_KEY = "url-relation-list"
    if DICT_KEY not in mb_artist:
        return []
    discogs_entry = [rel for rel in mb_artist[DICT_KEY] if rel["type"] == "discogs"]
    if discogs_entry:
        return discogs_entry[0]["target"].split("/")[-1]
    return None


def mb_get_group_releases(mb_artist, types=["Album"]):
    DICT_KEY = "release-group-list"
    if DICT_KEY not in mb_artist:
        return []
    for release in mb_artist[DICT_KEY]:
        try:
            release["first-release-date"] = du_parser.parse(release["first-release-date"])
        except (ValueError, OverflowError):
            continue
    if isinstance(types, str) and types.casefold() == "all":
        return mb_artist[DICT_KEY]
    elif isinstance(types, list):
        reqd_types = [req.casefold() for req in types]
        if "all" in reqd_types:
            return mb_artist[DICT_KEY]
        return [rel for rel in mb_artist[DICT_KEY] if rel["primary-type"].casefold() in reqd_types]
    return []


def mb_get_member_of(mb_artist, types=["Album"]):
    DICT_KEY = "artist-relation-list"
    if DICT_KEY not in mb_artist:
        return []
    member_of = []
    for group in [grp for grp in mb_artist[DICT_KEY] if grp["type"] == "member of band"]:
        group_dict = {}
        group_dict.update(group["artist"])
        if "begin" in group:
            group_dict["begin"] = du_parser.parse(group["begin"])
        if "end" in group:
            group_dict["end"] = du_parser.parse(group["end"])
        member_of.append(group_dict)
    return member_of


In [ ]:
jwhite_mb = mb_get_artist_by_id('3ae2fb37-8a23-429d-9920-bac811c4fc22')
mb_get_group_releases(jwhite_mb)

# discogs_get_artist_by_id(mb_get_discogs_id(mb_artist))

In [ ]:
mb_artist = mb_get_artist_by_id(jw_artist_id)
print(mb_get_discogs_id(mb_artist))

white_stripes = mb_get_artist_by_id("11ae9fbb-f3d7-4a47-936f-4c0a04d3b3b5")


In [8]:
import pandas as pd
pd.DataFrame(mb_get_group_releases(mb_artist, types=["album"]))

,id,type,title,first-release-date,primary-type,secondary-type-list
0,37f48931-e5e6-488f-a531-ad2db311158d,Album,Blunderbuss,2012-04-20,Album,NaN
1,67e28d5b-f68e-4349-8e43-76189c90be65,Album,Lazaretto,2014-06-06,Album,NaN
2,e084177d-10ad-4b06-af26-c0586272150a,Album,Boarding House Reach,2018-03-23,Album,NaN
3,d733d9b8-d5ea-4227-8629-5d7184f8f6ab,Compilation,Acoustic Recordings 1998–2016,2016-09-09,Album,[Compilation]
4,6c79efcc-9395-4a6f-b435-1e1abb68d705,Live,Acoustic Tour 2015,2016-06,Album,"[Compilation, Live]"
5,4ea6cb91-a86b-4b49-909a-e1d51fa8ddff,Live,Live at Third Man Records,2012-12,Album,[Live]
6,b0a28fe3-eed2-4008-ae77-ad1d7a5c7cf9,Live,Live From Bonnaroo 2014,2014-12,Album,[Live]
7,f67cda1f-d314-4dc0-88a6-3086c7926dc7,Live,Live at Third Man Records ||| Nashville & Cass...,2018-09,Album,[Live]


In [79]:
[band for band in mb_artist["artist-relation-list"] if band["type"] == "member of band"]
mb_artist

In [12]:
pd.DataFrame(mb_get_member_of(mb_artist))

,id,name,sort-name,begin,end,disambiguation
0,11ae9fbb-f3d7-4a47-936f-4c0a04d3b3b5,The White Stripes,"White Stripes, The",1997-02-15,2011-02-02,NaN
1,89025cbd-7ce3-4b5a-ab59-de2859d58b23,The Go,"Go, The",1998-02-15,1999-02-15,NaN
2,334bd887-9044-4c38-aebd-4baab851efb4,The Dead Weather,"Dead Weather, The",NaT,NaT,NaN
3,be407b02-f3e6-4ed5-9489-f8e5f0ab36dc,The Raconteurs,"Raconteurs, The",NaT,NaT,in Australia known as The Saboteurs
4,ab33138d-2f61-4bcb-89fb-9d0c56522e16,The Upholsterers,"Upholsterers, The",NaT,NaT,NaN
5,8b456736-fe99-4d33-ac69-cd267ee97b97,2 Star Tabernacle,2 Star Tabernacle,NaT,NaT,NaN
6,962b9083-46f3-4c5e-aa6a-c81cd01f5f7e,Jack White and The Bricks,"White, Jack and Bricks, The",NaT,NaT,NaN


# Last.FM

In [161]:
import requests
LFM_KEY_1 = "143c9cc433c0a23bb16ced806e8c40fb"
LFM_KEY = "6ca45596f0b24ec92a77c3281682cd68"
LFM_USER = "ianhellen"

LFM_ROOT = "http://ws.audioscrobbler.com{api}"
LFM_ARTIST = "/2.0/?method=artist.getinfo&artist={artist}&api_key={api_key}{mbid}&format=json"
LFM_TOP_ARTISTS = "/2.0/?method=user.gettopartists&user={user}&api_key={api_key}&limit={limit}&format=json"
LFM_TOP_ARTIST_ALBUMS = "/2.0/?method=artist.gettopalbums&artist={artist}&api_key={api_key}{mbid}&limit={limit}&format=json"
LFM_SIM_ARTIST = "/2.0/?method=artist.getsimilar&artist={artist}&api_key={api_key}&format=json"
import pandas as pd

def lfm_get_top_artists(user: str, api_key: str = LFM_KEY, limit: int = 100):
    last_fm_artists = LFM_ROOT.format(api=LFM_TOP_ARTISTS)
    resp = requests.get(last_fm_artists.format(
        user=user,
        api_key=LFM_KEY,
        limit=limit
    ))
    return resp.json()["topartists"]["artist"]

def lfm_get_top_artists_df(user: str, api_key: str = LFM_KEY, limit: int = 100):
    return pd.json_normalize(lfm_get_top_artists(user, api_key, limit))
    
def lfm_get_artist_info(artist: str,  api_key: str = LFM_KEY, mbid: str = None):
    mbid_opt = f"&mbid={mbid}" if mbid else ""
    url = LFM_ROOT.format(api=LFM_ARTIST)
    resp = requests.get(url.format(
        artist=artist,
        api_key=LFM_KEY,
        mbid=mbid_opt,
    ))
    artist_info = resp.json()
    return artist_info.get("artist", {})

def lfm_artist_similar(artist_info):
    similar_artists = {}
    for sim_artist in artist_info["similar"]["artist"]:
        similar_artists[sim_artist["name"]] = sim_artist
    return similar_artists
#     return artist_info.get("similar", {}).get("artist", {})
    
def lfm_artist_bio(artist_info):
    return artist_info.get("bio", {})
#     return pd.json_normalize(resp.json()["topartists"]["artist"])

def lfm_get_top_artist_albums(artist: str,  api_key: str = LFM_KEY, mbid: str = None, limit: int = 100):
    mbid_opt = f"&mbid={mbid}" if mbid else ""
    url = LFM_ROOT.format(api=LFM_TOP_ARTIST_ALBUMS)
    resp = requests.get(url.format(
        artist=artist,
        api_key=LFM_KEY,
        mbid=mbid_opt,
        limit=limit,
    ))
    album_info = resp.json()
    all_albums = album_info["topalbums"]["album"]
    return [album for album in top_albums if "mbid" in album]


def lfm_get_similar_artists(artist: str, api_key: str = LFM_KEY, num_artists=20):
    url = LFM_ROOT.format(api=LFM_SIM_ARTIST)
    similar_artists = {}
    resp = requests.get(url.format(
        artist=artist,
        api_key=api_key,
        limit=num_artists
    ))
    sim_resp = resp.json()
    if "similarartists" not in sim_resp:
        return {}
    for sim_artist in sim_resp["similarartists"]["artist"][:num_artists]:
        similar_artists[sim_artist["name"]] = sim_artist
    return similar_artists

### Create_artist

In [239]:
[t.name for t in artists_db2.artists.values() if not t.tracked]

['Ian Hellen',
 'Beirut',
 'Lana Del Rey',
 'Rafe Pearlman & Jonathan Plum',
 'Janelle Monáe',
 'The Kooks',
 'Mac DeMarco',
 'Los Lobos',
 'Bastille',
 'Erik Satie',
 'Flying Lotus',
 'Oh Wonder',
 'MF DOOM',
 'Purity Ring',
 '坂本龍一',
 'Lorde',
 'Richard H. Kirk',
 'Gorillaz',
 'Morcheeba',
 'Jóhann Jóhannsson',
 'Years & Years',
 "Dustin O'Halloran",
 'Damon Albarn',
 'Eno • Hyde',
 'Federico Aubele',
 'Neutral Milk Hotel',
 'Robert Rich',
 'Bruno Sanfilippo & Mathias Grassow',
 'Ludovico Einaudi',
 'The Rolling Stones',
 'Apparat',
 'Clem Leek',
 'Marian Hill',
 'Nils Frahm & Anne Müller',
 'Tei Shi',
 'Vladimir Ashkenazy',
 'Goldmund',
 'Julia Kent',
 'Peter Broderick',
 'Sneaker Pimps',
 'Algiers',
 'Deptford Goth',
 'Library Tapes',
 'Phantogram',
 'Pixies',
 'Cyberchump',
 'Fabrizio Paterlini',
 'Feathers',
 'Meg Myers',
 'Robin Guthrie & Harold Budd',
 'Sigur Rós',
 'Sylvain Chauveau',
 'Walk Off the Earth',
 'Edward Sharpe & The Magnetic Zeros',
 'Frank Zappa',
 'Keith Kenniff'

In [137]:
[a for a in init_art_select.options if a not in to_delete_select.options]
top_artists_df.head()

,mbid,url,playcount,image,name,streamable,@attr.rank,tracked
0,,https://www.last.fm/music/Ian+Hellen,155,"[{'size': 'small', '#text': 'https://lastfm.fr...",Ian Hellen,0,1,False
1,3ae2fb37-8a23-429d-9920-bac811c4fc22,https://www.last.fm/music/Jack+White,153,"[{'size': 'small', '#text': 'https://lastfm.fr...",Jack White,0,2,True
2,,https://www.last.fm/music/Beck,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Beck,0,3,True
3,ff95eb47-41c4-4f7f-a104-cdc30f02e872,https://www.last.fm/music/Brian+Eno,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Brian Eno,0,4,True
4,7dc8f5bd-9d0b-4087-9f73-dc164950bbd8,https://www.last.fm/music/Queens+of+the+Stone+Age,85,"[{'size': 'small', '#text': 'https://lastfm.fr...",Queens of the Stone Age,0,5,True


### Add artists to DB

In [141]:
zdb_root.artists_db.add_artists(top_artists_df.apply(create_artist, axis=1).array)

assert "Jack White" in zdb_root.artists_db
assert "3ae2fb37-8a23-429d-9920-bac811c4fc22" in zdb_root.artists_db

# Get Releases for Year

In [11]:
import pandas as pd
nme_url = "https://www.nme.com/releases/albums/{year}-calendar"

release_df = pd.read_html(nme_url.format(year="2019"), header=1)[0]

In [7]:
release_df

,Artist,Album
0,The Twilight Sad,It Won't Be Like This All The Time
1,18 January 2019,18 January 2019
2,Artist,Album
3,Deerhunter,Why Hasn't Everything Already Disappeared?
4,Future,The Wizrd
...,...,...
372,Tei Shi,La Linda
373,22 November 2019,22 November 2019
374,Artist,Album
375,Leonard Cohen,Thanks For The Dance


In [132]:
datetime.now().year

2020

## Wiki Releases

In [188]:
def get_wiki_releases
wiki_albums = "https://en.wikipedia.org/wiki/List_of_{year}_albums"

wiki_tbls = pd.read_html(wiki_albums.format(year=year))
wiki_release_df = pd.concat(wiki_tbls[2:-1], ignore_index=True)
len(wiki_tbls)

15

,Artist,Album,Genre,Label,Ref.,Ref,ReleaseDate
0,Legion of the Damned,Slaves of the Shadow Realm,NaN,Napalm,[1],NaN,2019-01-04
1,Born of Osiris,The Simulation,Progressive metalcore,Sumerian,[2],NaN,2019-01-11
2,Gnash,We,NaN,Atlantic,[3],NaN,2019-01-11
3,The Kentucky Headhunters,Live at the Ramblin' Man Fair,NaN,Alligator,[4],NaN,2019-01-11
4,Randy Houser,Magnolia,Country,Stoney Creek,[5],NaN,2019-01-11
...,...,...,...,...,...,...,...
654,Maruv,Hellcat Story,NaN,Warner Music Russia,[658],NaN,2019-11-29
655,New Riders of the Purple Sage,Thanksgiving in New York City,Country rock,Omnivore,[659],NaN,2019-11-29
656,Ozuna,Nibiru,NaN,"Aura Music, Sony Latin",[660],NaN,2019-11-29
657,Ozzy Osbourne,See You on the Other Side,NaN,Epic,[661],NaN,2019-11-29


In [189]:
wiki_release_df["Month"] = wiki_release_df["Release date"].str.extract(r"([^\d]+)")
wiki_release_df["Day"] = wiki_release_df["Release date"].str.extract(r"(\d+)", expand=True)
wiki_release_df["ReleaseDate"] = wiki_release_df.apply(lambda x: pd.to_datetime("-".join([str(x.Day), x.Month, str(year)])), axis=1)
wiki_release_df = wiki_release_df.drop(columns=["Day", "Month", "Release date", "Ref", "Ref."])
wiki_release_df.head()

,Artist,Album,Genre,Label,ReleaseDate
0,Legion of the Damned,Slaves of the Shadow Realm,NaN,Napalm,2019-01-04
1,Born of Osiris,The Simulation,Progressive metalcore,Sumerian,2019-01-11
2,Gnash,We,NaN,Atlantic,2019-01-11
3,The Kentucky Headhunters,Live at the Ramblin' Man Fair,NaN,Alligator,2019-01-11
4,Randy Houser,Magnolia,Country,Stoney Creek,2019-01-11


## NME Releases

In [31]:
from datetime import datetime
year = "2019"
def fetch_nme_releases(year):
    
    nme_cur_year_url = "https://www.nme.com/releases/albums"
    nme_past_yrs_url = "https://www.nme.com/releases/albums/{year}-calendar"
    if int(year) == datetime.now().year:
        url = nme_cur_year_url
    else:
        url = nme_past_yrs_url
    release_df = pd.read_html(url.format(year=year), header=1)[0]
    rel_date = pd.Timestamp(f"01/01/{year}")
    release_list = []
    for row in release_df.itertuples(index=False, name="Release"):
        if row[0].endswith(str(year)):
            rel_date = pd.Timestamp(row[0])
        elif row[0] == "Artist" and row[1] == "Album":
            continue
        else:
            release_list.append({
                "ReleaseDate": rel_date,
                "Artist": row[0],
                "Album": row[1]
                })
    
    release_df = pd.DataFrame(release_list)

releases2020 = fetch_nme_releases(2020)

### LFM Tests


In [44]:
rac_info = lfm_get_artist_info(artist="The Raconteurs")
print(rac_info)
rac_similar = lfm_get_similar_artists(artist="The Raconteurs", num_artists=40)

print({k: v["match"] for k, v in rac_similar.items() if float(v["match"]) >= 0.125})

albums = lfm_get_top_artist_albums(artist=rac_info["name"], mbid=rac_info["mbid"])
# top_albums = albums["topalbums"]["album"]
# [(album.get("name"), album.get("playcount"), album.get("mbid")) for album in top_albums if "mbid" in album]
pd.DataFrame(albums)

{'name': 'The Raconteurs', 'mbid': 'be407b02-f3e6-4ed5-9489-f8e5f0ab36dc', 'url': 'https://www.last.fm/music/The+Raconteurs', 'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'small'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'medium'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'large'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'extralarge'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': 'mega'}, {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png', 'size': ''}], 'streamable': '0', 'ontour': '0', 'stats': {'listeners': '1313219', 'playcount': '31946756'}, 'similar': {'artist': [{'name': 'Jack White', 'url': 'https://www.last.fm/music/Jack+White', 'im

,name,playcount,mbid,url,artist,image
0,Broken Boy Soldiers,12724439,0d6ea51b-d4bc-4e87-8db6-0f5110602594,https://www.last.fm/music/The+Raconteurs/Broke...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
1,Consolers of the Lonely,12997591,bdf59d5e-b1d0-36af-a9ac-5eea218b215a,https://www.last.fm/music/The+Raconteurs/Conso...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
2,"Steady, as She Goes",480990,81552b5f-3098-324a-821e-6f9c1e44d233,https://www.last.fm/music/The+Raconteurs/Stead...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
3,Salute Your Solution,54608,03695a6c-a355-430e-ae3d-4789d2d181fd,https://www.last.fm/music/The+Raconteurs/Salut...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
4,Broken Boy Soldier,114734,84a7a08a-7efa-456b-b1eb-f6c9fcf62641,https://www.last.fm/music/The+Raconteurs/Broke...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
5,Old Enough,33399,a8da547e-a6c7-4d38-9b74-33809449a244,https://www.last.fm/music/The+Raconteurs/Old+E...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
6,Hands,12223,86cd7a51-c3f7-4c1f-9f4c-26231ce51e10,https://www.last.fm/music/The+Raconteurs/Hands,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
7,"2006-03-22: Newcastle, UK",11923,272dad2d-1108-4984-be74-e3cde4753498,https://www.last.fm/music/The+Raconteurs/2006-...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...","[{'#text': '', 'size': 'small'}, {'#text': '',..."
8,Live at Third Man Records,14623,69804512-6f95-473b-a09b-803b2ece992a,https://www.last.fm/music/The+Raconteurs/Live+...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...
9,Zane Lowe BBC Session,4900,9d8894a9-011a-4029-9623-10a41691b689,https://www.last.fm/music/The+Raconteurs/Zane+...,"{'name': 'The Raconteurs', 'mbid': 'be407b02-f...",[{'#text': 'https://lastfm.freetls.fastly.net/...


In [ ]:
{k: v for k, v in rac_similar.items() if float(v["match"]) >= 0.25}

In [47]:
top_artists = lfm_get_top_artists(user=LFM_USER, limit=200)
top_artists_df = pd.json_normalize(top_artists)
top_artists_df.head()

,mbid,url,playcount,image,name,streamable,@attr.rank
0,,https://www.last.fm/music/Ian+Hellen,155,"[{'size': 'small', '#text': 'https://lastfm.fr...",Ian Hellen,0,1
1,3ae2fb37-8a23-429d-9920-bac811c4fc22,https://www.last.fm/music/Jack+White,153,"[{'size': 'small', '#text': 'https://lastfm.fr...",Jack White,0,2
2,,https://www.last.fm/music/Beck,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Beck,0,3
3,ff95eb47-41c4-4f7f-a104-cdc30f02e872,https://www.last.fm/music/Brian+Eno,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Brian Eno,0,4
4,7dc8f5bd-9d0b-4087-9f73-dc164950bbd8,https://www.last.fm/music/Queens+of+the+Stone+Age,85,"[{'size': 'small', '#text': 'https://lastfm.fr...",Queens of the Stone Age,0,5


In [61]:
zdb_root.artists_db

<__main__.RTArtists object at 0x000001F7E2191190 oid 0x1 in <Connection at 1f7e20f6a00>>

In [62]:
def create_artist(row):
    artist = RTArtist(name=row["name"], mbid=row["mbid"], lfm_url=row["url"])
    return artist


zdb_root.artists_db.add_artists(top_artists_df.apply(create_artist, axis=1).array)

In [35]:
top_artists_df[top_artist_df["name"] == a["image"][1]["#text"]

'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png'

In [ ]:
"3ae2fb37-8a23-429d-9920-bac811c4fc22"

In [32]:
import ipywidgets as widgets
from IPython.display import Image, display, Markdown
import pdb
def display_artist(selection):
#     with artist_output:
#         print(selection)
    if (
        selection is None
        or "new" not in selection
        #or not isinstance(selection["new"], int)
    ):
        return
    artist_idx = selection["new"]
    
    artist_rec = top_artists_df.iloc[artist_idx]
    artist_name = artist_rec["name"]
    artist_mbid = artist_rec["mbid"]
    artist_output.clear_output()
    with artist_output:
        display(Markdown(f"getting data..."))
    
    artist_info = lfm_get_artist_info(artist=artist_name, api_key=LFM_KEY, mbid=artist_mbid)
    do_artist = None
    if mbid:
        mb_artist = mb_get_artist_by_id(mbid)
        do_artist = discogs_get_artist_by_id(mb_get_discogs_id(mb_artist))

    sim_artists = lfm_get_similar_artists(artist=artist_name, api_key=LFM_KEY)
    bio = lfm_artist_bio(artist_info)
    new_artists, known_artists = get_new_sim_artists(top_artists_df, sim_artists)
    img_data = artist_rec.get("image", [])
    img_url = do_artist.data.get("thumb")
    if not img_url and img_data:
        if len(img_data) >= 2:
            img_url = img_data[1].get("#text", "")
        else:
            img_url = img_data[0].get("#text", "")

    artist_output.clear_output()
    with artist_output:
        display(Markdown(f"## {artist_name}"))
        display(Markdown(f"mbid: {artist_mbid}"))
        display(Markdown(bio.get("summary")))
        display(Image(url=img_url))
        display(Markdown(f"URL: {artist_rec['url']}"))
        if do_artist:
            display(Markdown("#### Aliases"))
            display(Markdown(", ".join([a.name for a in do_artist.aliases])))
            display(Markdown("#### Name variants"))
            display(Markdown(", ".join(do_artist.name_variations)))
        display(Markdown("#### Related Artists (new)"))
        display(Markdown(f"**{', '.join(new_artists)}**"))
        display(Markdown("#### Related Artists (known)"))
        display(Markdown(f"*{', '.join(known_artists)}*"))
    sel_new_art.options = new_artists
    sel_new_art.visible = bool(new_artists)
    move_to_selected_btn.visible = bool(new_artists)


def get_new_sim_artists(selected_artists: pd.DataFrame, sim_artists: dict):
    known_artists = set(selected_artists["name"].values)
    new_artists = sim_artists.keys() - known_artists
    known_artists = known_artists & sim_artists.keys()
    return new_artists, known_artists


def add_new_artists(btn_event):
    del btn_event
    for new_artist in sel_new_art.value:
        new_art_info = lfm_get_artist_info(artist=new_artist, api_key=LFM_KEY)
    new_artists[new_artist] = new_art_info
    all_new_art.options=sorted(new_artists.keys())


new_artists = dict()

artist_output = widgets.Output(layout={"border": "1px solid black"})
art_dict = {v: k for k,v in top_artists_df["name"].to_dict().items()}
selector = widgets.SelectMultiple(options=art_dict, 
                                  description="Select Artist",
                                  layout=widgets.Layout(width="50%", height="300px"),)
sel_new_art = widgets.SelectMultiple(options=[], 
                                     description="Select Artist",
                                     layout=widgets.Layout(width="300px", height="200px"),
                                     style={"description_width": "initial"},
                                     visible=False)
all_new_art = widgets.Select(options=[], 
                             description="Selected Artists",
                             layout=widgets.Layout(width="300px", height="300px"),
                             style={"description_width": "initial"})
add_btn = widgets.Button(description="Add to Tracked", visible=False)
sel_new_art.visible = False
add_btn.visible = False
add_btn.on_click(add_new_artists)

selector.observe(display_artist, names="value")
display(widgets.VBox([selector, artist_output,
                      widgets.HBox([widgets.VBox([sel_new_art, add_btn]), all_new_art])]))

In [95]:
artist_rec = top_artists_df.iloc[2]
artist_name = artist_rec["name"]
artist_mbid = artist_rec["mbid"]
do_artist = discogs_find_artist(artist_name)
sim_artists = lastfm_get_similar_artists(artist=artist_name, api_key=LAST_FM_KEY)

In [93]:
top_artists_df

,mbid,url,playcount,image,name,streamable,@attr.rank
0,,https://www.last.fm/music/Ian+Hellen,155,"[{'size': 'small', '#text': 'https://lastfm.fr...",Ian Hellen,0,1
1,3ae2fb37-8a23-429d-9920-bac811c4fc22,https://www.last.fm/music/Jack+White,103,"[{'size': 'small', '#text': 'https://lastfm.fr...",Jack White,0,2
2,,https://www.last.fm/music/Beck,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Beck,0,3
3,ff95eb47-41c4-4f7f-a104-cdc30f02e872,https://www.last.fm/music/Brian+Eno,99,"[{'size': 'small', '#text': 'https://lastfm.fr...",Brian Eno,0,4
4,7dc8f5bd-9d0b-4087-9f73-dc164950bbd8,https://www.last.fm/music/Queens+of+the+Stone+Age,84,"[{'size': 'small', '#text': 'https://lastfm.fr...",Queens of the Stone Age,0,5
...,...,...,...,...,...,...,...
195,be0cb2f2-71c3-4e95-a584-c19248951ff4,https://www.last.fm/music/DANGERDOOM,2,"[{'size': 'small', '#text': 'https://lastfm.fr...",DANGERDOOM,0,196
196,d4659efb-b8eb-4f03-95e9-f69ce35967a9,https://www.last.fm/music/David+Byrne,2,"[{'size': 'small', '#text': 'https://lastfm.fr...",David Byrne,0,197
197,0637b9fc-5864-46fe-9dcd-487c5c0d213d,https://www.last.fm/music/Deaf+Center,2,"[{'size': 'small', '#text': 'https://lastfm.fr...",Deaf Center,0,198
198,d2db1078-c170-4e22-b261-14f3ad60a690,https://www.last.fm/music/Devo,2,"[{'size': 'small', '#text': 'https://lastfm.fr...",Devo,0,199


In [41]:
last_fm_sim_artists = "http://ws.audioscrobbler.com/2.0/?method=artist.getsimilar&artist={artist}&api_key={api_key}&format=json"

similar_artists = top_artists_df.set_index("mbid")[["name"]].to_dict()["name"]
for _, row in top_artists_df.iterrows():
    print(row["name"])
    resp = requests.get(last_fm_sim_artists.format(
        artist=row["name"],
        api_key="143c9cc433c0a23bb16ced806e8c40fb",
        limit=10
    ))
    sim_resp = resp.json()
    if "similarartists" not in sim_resp:
        continue
    for sim_artist in sim_resp["similarartists"]["artist"][:20]:
        if "mbid" in sim_artist:
            similar_artists[sim_artist["mbid"]] = sim_artist["name"]
similar_artists

Ian Hellen
Jack White
Beck
Brian Eno
Queens of the Stone Age
Bruno Sanfilippo
Bonobo
Beirut
Eagles of Death Metal
Rafe Pearlman & Jonathan Plum
Lana Del Rey
Ólafur Arnalds
Janelle Monáe
David Bowie
The Chemical Brothers
Lou Reed
The Kooks
Cage the Elephant
Mac DeMarco
The Velvet Underground
Turin Brakes
Los Lobos
Max Richter
Bastille
Nils Frahm
The Strokes
Field Rotation
Zero 7
Hauschka
Erik Satie
Flying Lotus
Joywave
The Heavy
Blur
Django Django
Oh Wonder
Boards of Canada
MF DOOM
Ice Cube
Purity Ring
坂本龍一
Lorde
Scott Walker
Richard H. Kirk
Gorillaz
Morcheeba
Cake
Jóhann Jóhannsson
The Prodigy
The White Stripes
Years & Years
Arctic Monkeys
Dustin O'Halloran
elbow
Damon Albarn
Eno • Hyde
Federico Aubele
Neutral Milk Hotel
Robert Rich
Bomba Estéreo
Bruno Sanfilippo & Mathias Grassow
Ludovico Einaudi
Roger Eno
The Rolling Stones
Antonymes
Apparat
Clem Leek
Fripp & Eno
Marian Hill
Nils Frahm & Anne Müller
Tei Shi
Thievery Corporation
Vladimir Ashkenazy
Goldmund
Julia Kent
Peter Broderick
R

{'': 'Robin Guthrie & Harold Budd',
 '3ae2fb37-8a23-429d-9920-bac811c4fc22': 'Jack White',
 'ff95eb47-41c4-4f7f-a104-cdc30f02e872': 'Brian Eno',
 '7dc8f5bd-9d0b-4087-9f73-dc164950bbd8': 'Queens of the Stone Age',
 '8716f569-77ce-4044-a578-26c2f142c67a': 'Bruno Sanfilippo',
 '9a709693-b4f8-4da9-8cc1-038c911a61be': 'Bonobo',
 '0af78501-5647-4c18-9a0d-66ac8789e13b': 'Beirut',
 'de11b037-d880-40e0-8901-0397c768c457': 'Eagles of Death Metal',
 'b7539c32-53e7-4908-bda3-81449c367da6': 'Lana Del Rey',
 '6655955b-1c1e-4bcb-84e4-81bcd9efab30': 'Ólafur Arnalds',
 'ee190f6b-7d98-43ec-b924-da5f8018eca0': 'Janelle Monáe',
 '5441c29d-3602-4898-b1a1-b77fa23b8e50': 'David Bowie',
 '1946a82a-f927-40c2-8235-38d64f50d043': 'The Chemical Brothers',
 '9d1ebcfe-4c15-4d18-95d3-d919898638a1': 'Lou Reed',
 'f82f3a3e-29c2-42ca-b589-bc5dc210fa9e': 'The Kooks',
 'b41b38d4-ef3e-4f37-8c75-cfe9af999696': 'Cage the Elephant',
 'f2492c31-54a8-4347-a1fc-f81f72873bbf': 'Mac DeMarco',
 '94b0fb9d-a066-4823-b2ec-af1d324bcfc

In [46]:
# Remove unicode characters
import unicodedata
for artist_name in similar_artists.values():
    print(unicodedata.normalize('NFD', artist_name).encode('ascii', 'ignore').decode('utf-8'))

Robin Guthrie & Harold Budd
Jack White
Brian Eno
Queens of the Stone Age
Bruno Sanfilippo
Bonobo
Beirut
Eagles of Death Metal
Lana Del Rey
Olafur Arnalds
Janelle Monae
David Bowie
The Chemical Brothers
Lou Reed
The Kooks
Cage the Elephant
Mac DeMarco
The Velvet Underground
Turin Brakes
Los Lobos
Max Richter
Nils Frahm
The Strokes
Field Rotation
Zero 7
Hauschka
Erik Satie
Flying Lotus
Joywave
The Heavy
Blur
Django Django
Oh Wonder
Boards of Canada
MF DOOM & MF Grimm
Ice Cube
Purity Ring

Lorde
Scott Walker + Sunn O)))
Richard H. Kirk
Gorillaz
Morcheeba
Cake
Johann Johannsson
The Prodigy
The White Stripes
Years & Years
Arctic Monkeys
Dustin O'Halloran
Elbow
Damon Albarn
Federico Aubele
Neutral Milk Hotel
Robert Rich & Alio Die
Bomba Estereo
Ludovico Einaudi
Roger Eno
The Rolling Stones
Antonymes
Apparat
Clem Leek
Marian Hill
Tei Shi
Thievery Corporation
Vladimir Ashkenazy
Goldmund
Julia Kent
Peter Broderick
Rafael Anton Irisarri
Sneaker Pimps
Algiers
Deptford Goth
Ian Brown
Iggy Pop & Ja

In [65]:
top_artists = top_artists_df.set_index("mbid")[["name"]].to_dict()["name"]

In [67]:
norm_top_artists = {normalize(a) for a in top_artists.values()}

In [75]:
releases2019[releases2019["Artist"].str.lower().isin(norm_top_artists)]

,ReleaseDate,Artist,Album
19,2019-02-01,Beirut,Gallipoli
24,2019-02-01,Ian Brown,Ripples
118,2019-04-12,The Chemical Brothers,No Geography
120,2019-04-19,Cage the Elephant,Social Cues
149,2019-05-10,Mac DeMarco,Here Comes the Cowboy
168,2019-08-30,Lana Del Rey,Norman Fucking Rockwell!
180,2019-09-06,Iggy Pop,Free
203,2019-09-13,Pixies,Beneath The Eyrie
250,2019-10-11,Elbow,Giants of All Sizes
292,2019-11-15,Tei Shi,La Linda


In [191]:
# import persistent
# from persistent.list import PersistentList
# from persistent.mapping import PersistentMapping
# from BTrees.OOBTree import OOBTree

# from dataclasses import dataclass, field
# from datetime import datetime
# from enum import Enum, auto
# import uuid
# from typing import Dict, List, Any


# def _parse_enum(enum_type, name: str):
#     cf_members = {n.casefold(): v for n,v in enum_type.__members__.items()}
#     if name.casefold() in cf_members:
#         return cf_members[name.casefold()]
#     return None

# class ArtistRating(Enum):
#     Other = auto()
#     Favorite = auto()
#     Like = auto()
#     DontLike = auto()
#     Unknown = auto()

#     @classmethod
#     def parse(cls, name: str):
#         enum_val = _parse_enum(cls, name)
#         return enum_val if enum_val else cls.Other

# class ArtistType(Enum):
#     Other = auto()
#     Person = auto()
#     Group = auto()

#     @classmethod
#     def parse(cls, name: str):
#         enum_val = _parse_enum(cls, name)
#         return enum_val if enum_val else cls.Other
    

# class RTArtists(persistent.Persistent):
    
#     def __init__(self):
#         self.artists: OOBTree = OOBTree()
#         self.artist_ids: OOBTree = OOBTree()
            
#     def add_artist(self, artist):
#         self.artists[artist.name] = artist
#         if artist.mbid:
#             self.artist_ids[artist.mbid] = artist

#     def del_artist(self, artist):
#         if isinstance(artist, RTArtist):
#             key = artist.name
#         elif isinstance(artist, str):
#             key = artist
            
#         if key in self.artists:
#             del_artist = self.artists.pop(key)
#             if del_artist.mbid in self.artist_ids:
#                 self.artist_ids.pop(del_artist.mbid)
#         elif key in self.artist_ids:
#             del_artist = self.artist_ids.pop(key)
#             if del_artist.name in self.artists:
#                 self.artists.pop(del_artist.name)

#     def __getitem__(self, key):
#         if key in self.artists:
#             return self.artists[key]
#         if key in self.artist_ids:
#             return self.artist_ids[key]
#         return None
        
#     def __contains__(self, key):
#         return key in self.artists or key in self.artist_ids
    
#     def add_artists(self, artists):
#         for artist in artists:
#             self.add_artist(artist)

# @dataclass
# class RTArtist(persistent.Persistent):
#     name: str
#     sort_name: str = None
#     mbid: str = None
#     discogs_id: str = None
#     artist_type: ArtistType = ArtistType.Other
#     artist_rank: int = 500
#     favorite: bool = False
#     lfm_url: str = None
#     discogs_url: str = None
#     bio_summary: str = None
#     bio_content: str = None
#     tags: PersistentList = field(default_factory=PersistentList)
#     artist_rating: ArtistRating = ArtistRating.Unknown
#     releases: OOBTree = field(default_factory=OOBTree)
#     aliases: PersistentList = field(default_factory=PersistentList)
#     name_variations: PersistentList = field(default_factory=PersistentList)
#     members: OOBTree = field(default_factory=OOBTree)
#     member_of: OOBTree = field(default_factory=OOBTree)
#     similar: OOBTree = field(default_factory=OOBTree)
#     ext_data: OOBTree = field(default_factory=OOBTree)
#     img_url: str = None
#     tracked: bool = False
    
#     @property
#     def musicbrainz_url(self):
#         if self.mbid:
#             return f"https://musicbrainz.org/artist/{self.mbid}"
#         return None
    
#     def has_data(self, key):
#         return key.casefold() in self.ext_data
    
#     def _set_ext_data(self, key, data, overwrite=True):
#         if key.casefold() in ["lfm_artist", "discogs_artist", "mb_artist"]:
#             self.ext_data[key.casefold()] = data
#         else:
#             raise KeyError(f"Unknown data type {key}")
    
#     def populate_discogs_data(self, overwrite=False):
#         if not overwrite and self.has_data("discogs_artist"):
#             return
#         if self.discogs_id:
#             artist = discogs_get_artist_by_id(self.discogs_id)
#         else:
#             artist = discogs_find_artist(self.name)
#         if artist:
#             self._set_ext_data("discogs_artist", artist)
#             self.discogs_url = f"https://www.discogs.com/{artist.url}"
#             self.aliases = PersistentList([a.name for a in artist.aliases])
#             self.name_variations = PersistentList(artist.name_variations)
#             if artist.images and "uri" in artist.images[0]:
#                 self.img_url = artist.images[0]["uri"] 
    
#     def populate_mb_data(self, overwrite=False):
#         if not overwrite and self.has_data("mb_artist"):
#             return
#         if not self.mbid:
#             return
#         artist = mb_get_artist_by_id(self.mbid)
#         if not artist:
#             return
#         self.artist_type = ArtistType.parse(artist.get("type"))
#         self.sort_name = artist.get("sort-name")
#         self.discogs_id = mb_get_discogs_id(artist)
#         self._set_ext_data("mb_artist", artist)
#         membership = mb_get_member_of(artist)
#         for member in membership:
#             member_artist = RTArtist(name = member["name"], mbid=member["id"])
#             if self.artist_type == ArtistType.Person:
#                 self.member_of[member_artist.name] = member_artist
#             elif self.artist_type == ArtistType.Group:
#                 self.members[member_artist.name] = member_artist

#     def populate_lfm_data(self, overwrite=False):
#         if not overwrite and self.has_data("lfm_artist"):
#             return
#         artist = lfm_get_artist_info(self.name, LFM_KEY)
#         if not artist:
#             return
#         print(list(self.ext_data.keys()))
#         self._set_ext_data("lfm_artist", artist)
#         self.mbid = artist.get("mbid")
#         self.lfm_url = artist.get("url")
#         bio = lfm_artist_bio(artist)
#         if bio:
#             self.bio_summary = bio.get("summary")
#             self.bio_content = bio.get("content")
#         tag_list = artist.get("tags", {}).get("tag", [])
#         tag_names = [t_item["name"] for t_item in tag_list]
#         self.tags.append(tag_names)

#     def get_similar_artists(self, artist_db=None, overwrite=False):
#         if not overwrite and self.similar:
#             return
#         similar_artists_raw = lfm_get_similar_artists(
#             artist=self.name, api_key=LFM_KEY, num_artists=40
#         )
#         similar_artists = {
#             name: s_artist for name, s_artist 
#             in similar_artists_raw.items() 
#             if float(s_artist["match"]) >= 0.125
#         }
#         for a_name, s_artist in similar_artists.items():
#             if artist_db:
#                 if a_name in artist_db.artists:
#                     print("linking existing artist:", a_name)
#                     self.similar[a_name] = artist_db[a_name]
#                     continue
#             self.similar[a_name] = RTArtist(
#                 name=a_name, mbid=s_artist.get("mbid"), lfm_url=s_artist.get("url"))

#     def _add_mb_releases(self):
#         if "mb_artist" in self.ext_data:
#             releases = mb_get_group_releases(self.ext_data["mb_artist"])
#             for release in releases:
#                 self.releases[release.title] = release

#     def _create_mb_release(self, release) -> RTRelease:
#         rel_date = release.get("first-release-date")
#         release = RTRelease(
#             title = release.get("title"),
#             mbid = release.get("id"),
#             primary_type = release.get("primary_type"),
#             other_type = release.get("type"),
#             release_date = rel_date if isinstance(rel_date, datetime) else None,
#         )
#         release.set_ext_data("mb_release", release)
#         return release

# @dataclass
# class RTRelease(persistent.Persistent):
#     title: str
#     mbid: str = None
#     primary_type: str = None
#     other_type: str = None
#     release_date: datetime = None
#     ext_data: OOBTree = field(default_factory=OOBTree)

#     def set_ext_data(self, key, data, overwrite=True):
#         if key.casefold() in ["lfm_release", "discogs_release", "mb_release"]:
#             self.ext_data[key.casefold()] = data
#         else:
#             raise KeyError(f"Unknown data type {key}")

# Filter based on normalized sets

In [110]:
artist_names_norm = {art: set(normalize(art).split()) for art in top_artists.values()}
list(artist_names_norm.items())[:10]

[('Robin Guthrie & Harold Budd', {'&', 'budd', 'guthrie', 'harold', 'robin'}),
 ('Jack White', {'jack', 'white'}),
 ('Eno • Hyde', {'eno', 'hyde'}),
 ('Queens of the Stone Age', {'age', 'of', 'queens', 'stone', 'the'}),
 ('Bruno Sanfilippo & Mathias Grassow',
  {'&', 'bruno', 'grassow', 'mathias', 'sanfilippo'}),
 ('Bonobo', {'bonobo'}),
 ('Beirut', {'beirut'}),
 ('Eagles of Death Metal', {'death', 'eagles', 'metal', 'of'}),
 ('Lana Del Rey', {'del', 'lana', 'rey'}),
 ('Ólafur Arnalds', {'arnalds', 'olafur'})]

In [108]:
artist_names_norm = {art: set(normalize(art).split()) for art in top_artists.values()}
for _, release in releases2019.iterrows():
    for t_artist in artist_names_norm.values():
        if t_artist and not (t_artist - set(normalize(release["Artist"]).split())):
            print(release["Album"], release["Artist"])
            print(t_artist, set(normalize(release["Artist"]).split()))
            break

Gallipoli Beirut
{'beirut'} {'beirut'}
Ripples Ian Brown
{'brown', 'ian'} {'brown', 'ian'}
No Geography The Chemical Brothers
{'brothers', 'the', 'chemical'} {'brothers', 'the', 'chemical'}
Social Cues Cage the Elephant
{'elephant', 'cage', 'the'} {'elephant', 'cage', 'the'}
Here Comes the Cowboy Mac DeMarco
{'mac', 'demarco'} {'mac', 'demarco'}
Norman Fucking Rockwell! Lana Del Rey
{'lana', 'del', 'rey'} {'lana', 'del', 'rey'}
Free Iggy Pop
{'pop', 'iggy'} {'pop', 'iggy'}
Beneath The Eyrie Pixies
{'pixies'} {'pixies'}
Giants of All Sizes Elbow
{'elbow'} {'elbow'}
La Linda Tei Shi
{'tei', 'shi'} {'tei', 'shi'}


# Ultimate filter!

In [190]:
def matches_artist(release):
    for t_artist in artist_names_norm.values():
        if t_artist and not (t_artist - set(normalize(release["Artist"]).split())):
            return True
    return False
releases2019[releases2019.apply(matches_artist, axis=1)]

,ReleaseDate,Artist,Album
19,2019-02-01,Beirut,Gallipoli
24,2019-02-01,Ian Brown,Ripples
118,2019-04-12,The Chemical Brothers,No Geography
120,2019-04-19,Cage the Elephant,Social Cues
149,2019-05-10,Mac DeMarco,Here Comes the Cowboy
168,2019-08-30,Lana Del Rey,Norman Fucking Rockwell!
180,2019-09-06,Iggy Pop,Free
203,2019-09-13,Pixies,Beneath The Eyrie
250,2019-10-11,Elbow,Giants of All Sizes
292,2019-11-15,Tei Shi,La Linda


In [191]:
wiki_release_df[wiki_release_df.apply(matches_artist, axis=1)]

,Artist,Album,Genre,Label,ReleaseDate
50,Beirut,Gallipoli,NaN,4AD,2019-02-01
143,Apparat,LP5,Electronic,Mute,2019-03-22
185,The Chemical Brothers,No Geography,Electronica,Virgin EMI,2019-04-12
196,Cage the Elephant,Social Cues,"Alternative rock, garage rock, psychedelic rock",RCA,2019-04-19
199,The Rolling Stones,Honk,NaN,"Promotone BV, Universal",2019-04-19
248,Mac DeMarco,Here Comes the Cowboy,NaN,Mac's Record Label,2019-05-10
262,The Heavy,Sons,NaN,BMG,2019-05-17
275,Flying Lotus,Flamagra,"Jazz-funk, hip hop, electronic",Warp,2019-05-24
441,Lana Del Rey,Norman Fucking Rockwell!,"Soft rock, psychedelic rock, pop, folk rock","Polydor, Interscope",2019-08-30
457,Iggy Pop,Free,NaN,"Caroline, Loma Vista",2019-09-06


# Normalize data

In [58]:
import unicodedata
def normalize(str):
    return unicodedata.normalize('NFD', str).encode('ascii', 'ignore').decode('utf-8').lower()
    
norm_fav_artists = {normalize(a) for a in similar_artists.values()}
for _, row in releases2019.iterrows():
    if row["Artist"].lower() in norm_fav_artists:
        print(row)

ReleaseDate     2019-01-18 00:00:00
Artist             Sharon Van Etten
Album            Remind Me Tomorrow
ArtistNames    {Van, Sharon, Etten}
Name: 8, dtype: object
ReleaseDate    2019-02-01 00:00:00
Artist                      Beirut
Album                    Gallipoli
ArtistNames               {Beirut}
Name: 22, dtype: object
ReleaseDate           2019-02-01 00:00:00
Artist                             Broods
Album          Don't Feed the Pop Monster
ArtistNames                      {Broods}
Name: 24, dtype: object
ReleaseDate    2019-02-01 00:00:00
Artist                   Ian Brown
Album                      Ripples
ArtistNames           {Ian, Brown}
Name: 27, dtype: object
ReleaseDate    2019-02-08 00:00:00
Artist               Ariana Grande
Album                Thank U, Next
ArtistNames       {Grande, Ariana}
Name: 35, dtype: object
ReleaseDate    2019-02-08 00:00:00
Artist                Cass Mccombs
Album            Tip Of The Sphere
ArtistNames        {Mccombs, Cass}
Name: 36,

In [57]:
norm_fav_artists

{None}

In [35]:
releases2019[releases2019["Artist"].str.split(" ")]

,ReleaseDate,Artist,Album
208,2019-09-06,Iggy Pop,Free


In [19]:
releases2019["ArtistNames"] = releases2019.apply(lambda x: set(x["Artist"].split(" ")), axis=1)

In [24]:
top_artists_df[top_artists_df["name"].isin(releases2019["Artist"])]

,mbid,url,playcount,image,name,streamable,@attr.rank
7,0af78501-5647-4c18-9a0d-66ac8789e13b,https://www.last.fm/music/Beirut,58,"[{'size': 'small', '#text': 'https://lastfm.fr...",Beirut,0,8
10,b7539c32-53e7-4908-bda3-81449c367da6,https://www.last.fm/music/Lana+Del+Rey,47,"[{'size': 'small', '#text': 'https://lastfm.fr...",Lana Del Rey,0,11
14,1946a82a-f927-40c2-8235-38d64f50d043,https://www.last.fm/music/The+Chemical+Brothers,38,"[{'size': 'small', '#text': 'https://lastfm.fr...",The Chemical Brothers,0,15
17,b41b38d4-ef3e-4f37-8c75-cfe9af999696,https://www.last.fm/music/Cage+the+Elephant,32,"[{'size': 'small', '#text': 'https://lastfm.fr...",Cage the Elephant,0,18
18,f2492c31-54a8-4347-a1fc-f81f72873bbf,https://www.last.fm/music/Mac+DeMarco,32,"[{'size': 'small', '#text': 'https://lastfm.fr...",Mac DeMarco,0,19
70,921f2966-dfe6-4101-95fc-c8233d76f59f,https://www.last.fm/music/Tei+Shi,13,"[{'size': 'small', '#text': 'https://lastfm.fr...",Tei Shi,0,71
80,182d5d8a-94c7-4ef4-8222-a1838353a37b,https://www.last.fm/music/Ian+Brown,11,"[{'size': 'small', '#text': 'https://lastfm.fr...",Ian Brown,0,81
81,f37b3f31-b1f8-4b88-8cb5-b34f709b17d7,https://www.last.fm/music/Iggy+Pop,11,"[{'size': 'small', '#text': 'https://lastfm.fr...",Iggy Pop,0,82
84,b6b2bb8d-54a9-491f-9607-7b546023b433,https://www.last.fm/music/Pixies,11,"[{'size': 'small', '#text': 'https://lastfm.fr...",Pixies,0,85


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup


url = 'http://www.vru.gov.ua/act_list'


def get_
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
table = soup.find('table')

records = []
columns = []
for tr in table.findAll("tr"):
    ths = tr.findAll("th")
    if ths != []:
        for each in ths:
            columns.append(each.text)
    else:
        trs = tr.findAll("td")
        record = []
        for each in trs:
            try:
                link = each.find('a')['href']
                text = each.text
                record.append(link)
                record.append(text)
            except:
                text = each.text
                record.append(text)
        records.append(record)

columns.insert(1, 'Link')
df = pd.DataFrame(data=records, columns = columns)